In [ ]:
%matplotlib inline

In [ ]:
from libcomcat.search import get_event_by_id
from libcomcat.dataframes import get_pager_data_frame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import time
from scipy.optimize import curve_fit
from IPython.display import display

Working with tabular data in Python generally means working with the [pandas](https://pandas.pydata.org/) library. For the purposes of this discussion, pandas provides two useful data structures: A Series object (analogous to a dictionary), and a DataFrame object, which is analogous to an Excel spreadsheet, from which columns or rows can be extracted as Series objects.

DataFrames can be read in from delimited (comma, tab, etc.) text files, Excel files, database tables, or created from scratch. DataFrame objects provide a number of useful methods for data exploration and analysis including:

 - computation
 - statistics
 - plotting
 
In this notebook, we'll be reading, "cleaning", and exploring one data set.
 
## Contents
----
- [Reading Delimited Data](#Reading-Delimited-Data)
- [Cleaning Data](#Cleaning-Data)
  - [Indexing and Selecting](#Indexing-and-Selecting)
  - [Fixing Columns](#Fixing-Columns)
  - [Extracting Data From Columns](#Extracting-Data-From-Columns)
  - [Subsetting Data](#Subsetting-Data)
  - [Removing Duplicates](#Removing-Duplicates)
  - [Merging DataFrames](#Merging-DataFrames)
- [Exploring Data](#Exploring-Data)
  - [Comparing Observed and Predicted](#Comparing-Observed-and-Predicted)

## Reading Delimited Data

pandas can read data from a number of formats:

 - Comma* separated values (CSV)
 - Excel spreadsheets
 - Hierarchical Data Format (HDF) file
 - Relational databases
 
\*For reading CSV files, pandas provides a function called *read_csv()*. Despite its name, this function is not limited to reading text files with columns separated by commas. The delimiter can be just about anything - tabs are very common, as are pipes ("|"), as we see below. 



This data set was extracted from the Hydra QA database, and represents USGS National Earthquake Information Center (NEIC) analyst reports collected on earthquake impacts. There are a few things to note about this file that we will have to deal with before doing any analysis with it. This is known as "data cleaning".

 - As noted above the delimiter is a "|" character.
 - There are spaces around the column names
 - The last three lines of the file do not match the format in the rest of the file
 - Most of the data of interest is contained in the "spassportentry" column, and consists of columns separated by whitespace.

I've written two functions below that allow us to quickly see the beginning and ending lines of a file, similar to the *head* and *tail* commands available on OSX and Linux.

In [ ]:
def head(filename, nlines):
    with open(filename, 'rt') as f:
        for i in range(0,nlines):
            print('*'+f.readline()+'*')

In [ ]:
def tail(filename, nlines):
    lines = open(filename, 'rt').readlines()
    for i in range(-nlines,-1):
        print('*'+lines[i]+'*')

In [ ]:
impact_file = '../data/hydra_impact_comments.txt'
head(impact_file,5)

In [ ]:
tail(impact_file,3)

You can look at the documentation for read_csv [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), or by executing "pd.read_csv?" in a code cell.

Note that there are several options we can use to help ourselves out right off the bat:

 - the *parse_dates* option will allow us to read in the "ot" (origin time) column as a pandas Timestamp *object*, not just a string.
 - While we could read in the columns from the file and then fix the extra spaces, it's easier to use the *skiprows* option to skip the first line and provide our own column names.
 - We can use the *skipfooter* option to ignore those final three problematic lines.

In [ ]:
columns = ['hydra_id', 'hydra_time', 'hydra_magnitude', 'hydra_command', 'passport_entry']
hydra_impacts = pd.read_csv(impact_file, names=columns, engine='python',
                            parse_dates=['hydra_time'], skiprows=1, 
                            skipfooter=3, delimiter='|')
hydra_impacts.head()

Let's confirm that our timestamp parsing worked, by asking a question about *duration* of the data set.

In [ ]:
hydra_impacts['hydra_time'].max() - hydra_impacts['hydra_time'].min()

## Cleaning Data

### Fixing Columns
Next, let's convert that hydra ID to a ComCat-compatible event ID by prepending 'us' to each ID, and lowercasing the column while we're at it. Fortunately, pandas provides a number of string methods that can operate on all elements of an array (column, row, etc.) These are well-described [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#string-methods). Many of these functions should be familiar to anyone who has used the methods of the standard Python *str* object. 

It turns out that some of these hydra_id values have extraneous spaces in them as well. You can chain multiple string operations together, as seen below with lower() and strip(). 

Finally, it turns out that there are [multiple ways](https://stackoverflow.com/questions/20025882/add-a-string-prefix-to-each-value-in-a-string-column-using-pandas) to prepend a string to the values in a column. Once again StackOverflow comes to the rescue. This was the most syntactically straighforward solution.

In [ ]:
hydra_impacts['hydra_id'] = hydra_impacts['hydra_id'].str.lower().str.strip()
hydra_impacts['hydra_command'] = hydra_impacts['hydra_command'].str.strip()
hydra_impacts['hydra_id'] = 'us' + hydra_impacts['hydra_id'].astype(str)
hydra_impacts.head()

In this context, seeing magnitude reported beyond 1 significant digit is not useful. Let's round those values to the nearest tenth of a magnitude unit.

In [ ]:
hydra_impacts['hydra_magnitude'] = hydra_impacts['hydra_magnitude'].round(decimals=1)
hydra_impacts.head()

### Indexing and Selecting

You may be wondering at this point what the unlabeled column at the left of our DataFrame is for. This is an *index* column, and the values in it are used as labels for the rows. By default, pandas assigns integer labels for the index, but this index could be any kind of variable. In our situation, the most logical candidate for an index is the hydra_id.

In [ ]:
tmp_impacts = hydra_impacts.set_index('hydra_id')
tmp_impacts.head()

In [ ]:
# selecting single event by label (.loc) # this returns a series
tmp_impacts.loc['usc0004ktb']

In [ ]:
# selecting multiple events by label (.loc)
tmp_impacts.loc[['usc0004ktb','usc0005idz']]

In [ ]:
# selecting a column from multiple events 
tmp_impacts.loc[['usc0004ktb','usc0005idz'],'passport_entry']

In [ ]:
# selecting multiple events by position
tmp_impacts.iloc[0:2]

In [ ]:
# selecting single row by position
tmp_impacts.iloc[0]

In [ ]:
# selecting by row and column positions
tmp_impacts.iloc[0:2,[1,3]]

One of the most common use cases in our experience, however, is sub-selecting a DataFrame based on criteria in one of more colums. 

If you recall from numpy, you select elements of an array most efficiently by doing:

```x = np.arange(1,6)```

returns: array([1, 2, 3, 4, 5])

```x > 3```

returns: array([False, False, False,  True,  True], dtype=bool)


```x[x > 3]``` 

returns: array([4, 5])


The same sort of selection syntax is possible in pandas:

In [ ]:
tmp_impacts[tmp_impacts['hydra_time'] > pd.Timestamp('2020-06-01')]

You can also make complex selections - for example, we might want to see all events in 2019 that are greater than M6.0. If you have multiple selection criteria, you can split those criteria into different arrays and combine them with boolean operators.

In [ ]:
start_2019 = tmp_impacts['hydra_time'] > pd.Timestamp('2019-01-01')
end_2019 = tmp_impacts['hydra_time'] > pd.Timestamp('2019-12-31 23:59:59')
mag_6plus = tmp_impacts['hydra_magnitude'] > 6.0
tmp_impacts[start_2019 & end_2019 & mag_6plus].head()

### Extracting Data From Columns
Now for the hard part - we need to parse the impact data out of the passport_entry column. To handle this, we'll need to iterate over the rows of the dataframe, pull apart the passport_entry column, and create a new DataFrame. You can construct new DataFrame objects in a number of different ways - we'll create a list of Series objects and pass that to the constructor.

In [ ]:
IMPACT_COLUMNS = ['LossExtent',
                  'EffectType',
                  'LossQuantifier',
                  'LossValue',
                  'Location',
                  'CollectionSource',
                  'database_id',
                  'comment']
rows = []
for idx, hydra_row in hydra_impacts.iterrows():
    passport_parts = hydra_row['passport_entry'].split()
    # make a dictionary out of the passport entry
    pdict = dict(zip(IMPACT_COLUMNS, passport_parts))
    new_row_dict = hydra_row.to_dict()
    new_row_dict.update(pdict)
    rows.append(new_row_dict)
    
impacts = pd.DataFrame(rows)
# we need to change the data type of the LossValue column to an integer
impacts['LossValue'] = impacts['LossValue'].astype(np.int32)
impacts.head()

We don't need the passport_entry column anymore, so we use the *drop* method to get rid of it.

In [ ]:
impacts.drop('passport_entry', axis='columns', inplace=True)
impacts.head()

### Subsetting Data
Because we're only interested in fatalities due to shaking for the purpose of this analysis, let's extract just those rows.

In [ ]:
condition1 = impacts['LossExtent'] == 'Deaths'
condition2 = impacts['EffectType'] == 'Shaking'
condition3 = impacts['hydra_command'] == 'PubFlagsAddImpact'
deaths = impacts[condition1 & condition2 & condition3]
deaths.head()

### Removing Duplicates

You may notice that we have multiple fatality entries for some events. If this were a *true* analysis, we would do something clever and appropriate to determine the correct number of fatalities. Here we'll just choose the maximum value for each event, using an answer found from [StackOverflow](https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-count-in-groups-using-groupby)*. We'll also re-sort the data in time ascending order once we're finishing removing duplicates.

*Google and stack overflow are going to be your friends.

In [ ]:
#idx = deaths.groupby(['hydra_id'])['LossValue'].transform(max) == deaths['LossValue']
#newdeaths = deaths[idx]
newdeaths = deaths.sort_values('LossValue', ascending=False).drop_duplicates(['hydra_id'])
newdeaths.sort_values('hydra_time', inplace=True)
newdeaths.head()

Suppose that we wanted to see the distribution of magnitudes for the dataset as a bar chart. We could extract the number of occurrences of unique values using the `value_counts` method. Here we apply `value_counts` to the hydra_magnitude column. This returns a series with the first column being the unique value and the second column showing the number of occurrences.


In [ ]:
impact_data = hydra_impacts.drop_duplicates(subset='hydra_id')
magnitudes = impact_data['hydra_magnitude'].value_counts()
magnitudes

We can see that each unique magnitude has an associated number of occurrences; if we plot this as a bar chart, it is going to be very hard to decipher the data.

In [ ]:
plt.figure(figsize=(12, 5))
ax = magnitudes.plot.bar()
ystr = ax.set_ylabel('Number of occurrences', fontsize=18)
xstr = ax.set_xlabel('Magnitude', fontsize=18)
ticks = ax.set_xticklabels(magnitudes.index, rotation=60, fontsize=10) 
plt.show()

It would be much easier to evaluate this data if it were in bins. We can redo the value count with a set number of bins.

In [ ]:
magnitudes = impact_data['hydra_magnitude'].value_counts(bins=10)
magnitudes

Notice, that the bins are not sorted. They are instead in order of occurrence. We can sort this before plotting using the sort method.

In [ ]:
sorted_magnitudes = magnitudes.sort_index()
sorted_magnitudes

In [ ]:
plt.figure(figsize=(12, 5))
ax = sorted_magnitudes.plot.bar()
ystr = ax.set_ylabel('Number of occurrences', fontsize=18)
xstr = ax.set_xlabel('Magnitude', fontsize=18)
xticks = ax.set_xticklabels(sorted_magnitudes.index,
                           rotation=60, fontsize=14) 
yticks = ax.set_yticklabels(np.arange(0, 140, 20), fontsize=14) 
plt.show()

### Merging DataFrames

Let's say we want to compare these *observed* fatalities with the *predicted* fatalities from PAGER. We can extract those fatalities from ComCat using the libcomcat tools. It takes about 90 seconds to download this data, so we'll load it from a pre-fetched file here.

In [ ]:
expframe = pd.read_csv('../data/pager_exposures_hydra.csv')
expframe.head()

If you want to run the cell below, note that you'll be overriding the DataFrame *expframe* we read in above. Even if you don't run this cell, you should examine it carefully to notice some new things:

 - We are iterating over rows using the DataFrame *iterrows()* method, which returns the index and a reference to the Series representing the row of data.
 - We are also using two functions from the libcomcat library:
   - *get_event_by_id()* - This returns a DetailEvent object, from which you can extract *authoritative* event information.
   - *get_pager_data_frame()* This returns (in this case) a DataFrame with one row of PAGER exposure/loss information, or None if no PAGER results were created for the event.
 - We're also demonstrating the use of the DataFrame *to_csv()* method. Saving DataFrames to CSV or Excel is pretty straightforward, so we'll just point users to the documentation for [to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and [to_excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html#pandas.DataFrame.to_excel)
 
#### To re-create the *expframe* variable:

 - Change the type of the cell below from "Markdown" to "Code"
 - Remove the leading/trailing triple backticks \`\`\`
 - Execute the cell

```
# ---------------------------------------------------------
exprows = []
t1 = time.time()
for idx, row in newdeaths.iterrows():
    detail = get_event_by_id(row['hydra_id'])
    expdict = {}
    # fill in columns with ComCat authoritative values
    expdict['hydra_id'] = row['hydra_id']
    expdict['eventid'] = detail.id
    expdict['time'] = pd.to_datetime(detail.time)
    expdict['latitude'] = detail.latitude
    expdict['longitude'] = detail.longitude
    expdict['depth'] = detail.depth
    expdict['magnitude'] = detail.magnitude
    
    # now retrieve the PAGER exposure data for this event
    expframe = get_pager_data_frame(detail, get_losses=True)
    if expframe is None:
        msg = (f"event {row['hydra_id']} "
               f"Date: {row['hydra_time']} "
               f"Magnitude: {row['hydra_magnitude']}")
        print(f'No PAGER information found for {msg}. Skipping.')
        continue
    exprow = expframe.iloc[0]
    expdict['predicted_deaths'] = exprow['predicted_fatalities']
    expdict['mmi5'] = exprow['mmi5']
    expdict['mmi6'] = exprow['mmi6']
    expdict['mmi7'] = exprow['mmi7']
    expdict['mmi8'] = exprow['mmi8']
    expdict['mmi9'] = exprow['mmi9']
    expdict['mmi10'] = exprow['mmi10']
    exprows.append(expdict)
expframe = pd.DataFrame(exprows)
t2 = time.time()
print(f'Elapsed time: {t2-t1:.1f} seconds')
expframe.head()
expframe.to_csv('../data/pager_exposures_hydra.csv', index=False)
# ---------------------------------------------------------
```

Now we need to merge the *observed* fatality dataframe we've created with the *predicted* fatality/exposure dataset we've extracted from ComCat. pandas DataFrame objects provide us with the *merge* method, which is described in detail [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [ ]:
expdeaths = newdeaths.merge(expframe, on='hydra_id')
expdeaths.head()

## Exploring Data

### Comparing Observed and Predicted

We can now inspect the resulting DataFrame and see that we have all of the columns from both input DataFrames. At this point, the data has been effectively cleaned, and we can begin doing some simple data exploration and analysis.

The PAGER project calculates estimated fatalities for an event based on a ShakeMap input and country-specific loss models. When we want to get a visual sense of how the models are performing compared to observations of fatalities, it is useful to compare them within the ranges defined for PAGER alert levels, which are:

 - Green: 0 fatalities (no response needed)
 - Yellow: 1-99 fatalities (local/regional response)
 - Orange: 100-999 fatalities (national response)
 - Red: 1000+ fatalities (international response)
 
Our assumption is, for example, that the ground *response* to an earthquake with 25 fatalities will be about the same as one with 50 fatalities. So, we make a figure that demonstrates how often our predictions fall within the observed range, as with the plot below.

#### Sidebar: "A value is trying to be set on a copy of a slice from a DataFrame"

You may see this excessively passive voice error coming from pandas at some point, and wonder what the heck it means. 

When using pandas indexing to select a row or column from a DataFrame, what you get back is a *reference* to the same data that is in the original DataFrame. If you then try to *modify* this extracted row or column, you'll get the above error, along with a link to [this page](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy).

A well-written page describing what's going on here, and how to solve it:

[https://www.dataquest.io/blog/settingwithcopywarning/](https://www.dataquest.io/blog/settingwithcopywarning/)

We bring this up here because of the following lines of code in the cell below:

```
observed = expdeaths['LossValue'].to_numpy()
predicted = expdeaths['predicted_deaths'].to_numpy()
observed[observed == 0] = 0.01
predicted[predicted == 0] = 0.01
```

Adding the *to_numpy()* method to convert the observed and predicted fatality columns to numpy arrays explicitly makes copies of those two columns, thus avoiding the warning and doing (in this case) what we want.

In [ ]:
ALPHA = 0.75
GREEN = (0/255,176/255,79/255,ALPHA)
YELLOW = (255/255,255/255,0/255,ALPHA)
ORANGE = (255/255,153/255,0/255,ALPHA)
RED = (255/255,0/255,0/255,ALPHA)
fig = plt.figure(figsize=(10,10));
plt.loglog([0.001,1e7], [0.001,1e7],'w.');
ax = fig.axes[0]
errorboxes = []
rgreen = Rectangle((0.001,0.001),1,1, facecolor=GREEN, edgecolor='black');
ryellow = Rectangle((1,1),100,100, facecolor=YELLOW, edgecolor='black');
rorange = Rectangle((100,100),1000,1000, facecolor=ORANGE, edgecolor='black');
rred = Rectangle((1000,1000),1e7,1e7, facecolor=RED, edgecolor='black');
ax.add_patch(rgreen);
ax.add_patch(ryellow);
ax.add_patch(rorange);
ax.add_patch(rred);
observed = expdeaths['LossValue'].to_numpy()
predicted = expdeaths['predicted_deaths'].to_numpy()
observed[observed == 0] = 0.01
predicted[predicted == 0] = 0.01
plt.loglog(observed, predicted, 'b.');
xlocs, _ = plt.xticks();
ylocs, _ = plt.yticks();
xlocs = [0.001, 1, 10, 100, 1000, 1e5, 1e7]
ylocs = [0.001, 1, 10, 100, 1000, 1e5, 1e7]
newlabels = ['0', '1', '10', '$10^2$', '$10^3$', '$10^5$', '$10^7$']
plt.xticks(xlocs, newlabels);
plt.yticks(ylocs, newlabels);
plt.axis([0.001,1e7, 0.001, 1e7]);
plt.title('Observed Fatalities vs PAGER Predicted');
plt.text(0.01, 1e6,f'N = {len(predicted)}', fontsize=16);
plt.xlabel('Observed Fatalities');
plt.ylabel('Predicted Fatalities');

*NB: Note here that our sample size is only 51 events, so take this "analysis" with a grain of salt.*

You could possibly infer from the above plot that the PAGER system may:
 - underpredict events that have 0 observed fatalities
 - do a reasonable job at predicting "Yellow" alert events
 - overpredict events in the "Orange" alert range